In [ ]:
%matplotlib inline
import numpy as np
import torch
import copy
import pickle
from tqdm import tqdm

In [ ]:
import sklearn.datasets
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import sys

In [ ]:
from sklearn.datasets import fetch_california_housing
california_housing = fetch_california_housing(as_frame=True)

In [ ]:
california_housing.frame.head()

In [ ]:
X = california_housing.data
Y = california_housing.target

In [ ]:
# reorder the features
X = X[['Longitude', 'Latitude', 'AveOccup',  'Population', 'MedInc', 'HouseAge', 'AveRooms', 'AveBedrms']]

In [ ]:
features = list(X.columns)
print(features)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Fit a linear regression model to set the bias b\
Or equivalently, set b = Y_train.mean()



In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, Y_train)
b = model.intercept_
print(b)

In [ ]:
mask1 = [True, True, False, False, False, False, False, False]
mask2 = [True, True, True, True, True, False, False, False]
mask3 = [False, False, True, True, True, True, True, True]
mask4 = [False, False, False, False, False, True, True, True]

In [ ]:
def func1(X_train, Y_train, w, b, mask = mask1):
    X_masked = X_train.copy()
    X_masked[:, ~np.array(mask)] = 0
    predictions = np.dot(X_masked, w) + b
    mse_loss = np.mean((Y_train - (np.dot(X_masked, w) + b))**2)
    grad_w = 2 * X_masked.T @ (predictions - Y_train) / len(Y_train)
    return mse_loss, grad_w


In [ ]:
def func2(X_train, Y_train, w, b, mask = mask2):
    X_masked = X_train.copy()
    X_masked[:, ~np.array(mask)] = 0
    predictions = np.dot(X_masked, w) + b
    mse_loss = np.mean((Y_train - (np.dot(X_masked, w) + b))**2)
    grad_w = 2 * X_masked.T @ (predictions - Y_train) / len(Y_train)
    return mse_loss, grad_w

In [ ]:
def func3(X_train, Y_train, w, b, mask = mask3):
    X_masked = X_train.copy()
    X_masked[:, ~np.array(mask)] = 0
    predictions = np.dot(X_masked, w) + b
    mse_loss = np.mean((Y_train - (np.dot(X_masked, w) + b))**2)
    grad_w = 2 * X_masked.T @ (predictions - Y_train) / len(Y_train)
    return mse_loss, grad_w

In [ ]:
def func4(X_train, Y_train, w, b, mask = mask4):
    X_masked = X_train.copy()
    X_masked[:, ~np.array(mask)] = 0
    predictions = np.dot(X_masked, w) + b
    mse_loss = np.mean((Y_train - (np.dot(X_masked, w) + b))**2)
    grad_w = 2 * X_masked.T @ (predictions - Y_train) / len(Y_train)
    return mse_loss, grad_w

# MGDA implementation

In [ ]:
def find_sz(a,b):
  # find the minimizer of ax^2 + 2bx, where 0<=x<=1 (x is step size)

  # check that a is non negative
  if a < 0 :
    print("error in solving step size")
    return -1
  if a == 0:
    if b >=0 :
      sz = 0
    else:
      sz = 1
    return sz

  axis = - b * 1.0 / a
  if axis < 0 :
    sz = 0
  elif axis > 1:
    sz = 1
  else:
    sz = axis
  return sz

In [ ]:
# write a FW procedure to solve for the min norm element
def FW_solve_w(U, rounds=10, lambda_0=None):
  # Frank-Wolfe exact step size, with warm start


  n,d = U.shape
  lbd = 1.0/n * np.ones(n)

  if lambda_0 is not None:
    lbd = lambda_0

  # precomputing G here is actually better since it can be reused in the loop,
  # otherwise matrix/matrix multiplication (UU^T) is not efficient
  G = np.array(U.dot(U.T))

  for t in range(rounds):
    v = G.dot(lbd)
    idx_min = np.argmin(v)
    d = np.zeros(n,dtype=float)
    d[idx_min] = 1

    # find the best sz by solving a quadratic problem
    delta = d - lbd
    a = delta.dot(G.dot(delta))
    b = delta.dot(v)
    sz = find_sz(a,b)
    if sz < 0:
      sys.exit("error in running FW for solving QP")

    lbd = (1-sz) * lbd + sz * d
    if sz == 0:
      # print("it takes ith round:",t)
      break



  return U.T.dot(lbd), lbd

# Single trial (i.e. seed)

## Train with MGDA

In [ ]:
np.random.seed(1)
w_0 = np.random.rand(8)

In [ ]:
lr = 0.1
epoch = 500

In [ ]:
w = w_0.copy()
b = b
list_losses = []
for epoch in tqdm(range(epoch)):
    losses = []
    grads = []
    for i in range(4):
        loss, grad = eval(f"func{i+1}")(X_train, Y_train, w, b)
        losses.append(loss)
        grads.append(grad)

    grads = np.array(grads)
    common_grad, lbd = FW_solve_w(grads)

    w = w - lr * common_grad
    list_losses.append(losses)

In [ ]:
w_store = w.copy()

In [ ]:
mgda_list_losses = list_losses.copy()

In [ ]:
print(mgda_list_losses[-1])

In [ ]:
with open('chain_seed1_MGDA.pickle','wb') as f:
    pickle.dump(mgda_list_losses,f)

## Train/Refine with RP-MGDA

In [ ]:
np.random.seed(1)
w_0 = np.random.rand(8)

In [ ]:
lr = 0.01
epoch = 5 # 5 is for refine

In [ ]:
%time

# w = w_0.copy(), if training from start

# For refining MGDA solution, use following
w = w_store

b = b
list_losses = []
for epoch in tqdm(range(epoch)):
    losses = []
    grads = []
    for i in range(4):
        loss, grad = eval(f"func{i+1}")(X_train, Y_train, w, b)
        if epoch == 5:
            print(grad)
        losses.append(loss)
        grads.append(grad)

    # We do MGDA on theta_1 w[0:2] wrt (f1,f2)
    f1_grad1 = grads[0][0:2]
    f2_grad1 = grads[1][0:2]
    common_g1 = FW_solve_w(np.array([f1_grad1, f2_grad1]))[0]

    # We do MGDA on theta_2 w[2:5] wrt (f2,f3)
    f2_grad2 = grads[1][2:5]
    f3_grad2 = grads[2][2:5]
    common_g2 = FW_solve_w(np.array([f2_grad2, f3_grad2]))[0]

    # We do MGDA on theta_3 w[5:8] wrt (f3,f4)
    f3_grad3 = grads[2][5:8]
    f4_grad3 = grads[3][5:8]
    common_g3 = FW_solve_w(np.array([f3_grad3, f4_grad3]))[0]

    w[0:2] = w[0:2] - lr * common_g1
    w[2:5] = w[2:5] - lr * common_g2
    w[5:8] = w[5:8] - lr * common_g3

    list_losses.append(losses)

In [ ]:
rpmgda_list_losses = list_losses.copy()

In [ ]:
with open('chain_seed1_RPMGDA.pickle','wb') as f:
    pickle.dump(rpmgda_list_losses,f)

# Plot

## After running 20 trials and saving the results
With the 40 pickle files ready (20 seeds), we can run the following

In [ ]:
import pickle

mgda_results = {}
rpmgda_results = {}

for seed in range(1, 21):
  with open(f'chain_seed{seed}_MGDA.pickle', 'rb') as f:
    mgda_results[seed] = pickle.load(f)
  with open(f'chain_seed{seed}_RPMGDA.pickle', 'rb') as f:
    rpmgda_results[seed] = pickle.load(f)

## Mean and Stdev plots

This code iterates over the four losses, calculates the mean and standard deviation of the losses across the 20 seeds for both MGDA and RPMGDA, and plots the means with shaded regions representing the standard deviations. Each loss is plotted in a separate subplot, with labels and legends for clarity.

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(20, 4))
loss_names = ['MSE ($f_1$)', 'MSE ($f_2$)', 'MSE ($f_3$)', 'MSE ($f_4$)']

for i, ax in enumerate(axes.flat):
    mgda_losses = [np.array(mgda_results[seed])[:, i] for seed in range(1, 21)]
    rpmgda_losses = [np.array(rpmgda_results[seed])[:, i] for seed in range(1, 21)]

    mean_mgda = np.mean(mgda_losses, axis=0)
    std_mgda = np.std(mgda_losses, axis=0)
    mean_rpmgda = np.mean(rpmgda_losses, axis=0)
    std_rpmgda = np.std(rpmgda_losses, axis=0)

    ax.plot(mean_mgda, label='MGDA')
    ax.fill_between(range(len(mean_mgda)), mean_mgda - std_mgda, mean_mgda + std_mgda, alpha=0.2)
    ax.plot(mean_rpmgda, label='RPMGDA')
    ax.fill_between(range(len(mean_rpmgda)), mean_rpmgda - std_rpmgda, mean_rpmgda + std_rpmgda, alpha=0.2)

    ax.set_xlabel('Iteration',fontsize=18)
    ax.set_ylabel(loss_names[i],fontsize=18)
    ax.tick_params(axis='both', labelsize=12)
    ax.legend(fontsize=16)

plt.tight_layout()
plt.show()